# Regression Week 5:
## Feature selection and LASSO

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

In [2]:
import graphlab

In [3]:
sales = graphlab.SFrame('kc_house_data.gl/')

[INFO] 1452910312 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_FILE to /usr/local/lib/python2.7/dist-packages/certifi/cacert.pem
1452910312 : INFO:     (initialize_globals_from_environment:282): Setting configuration variable GRAPHLAB_FILEIO_ALTERNATIVE_SSL_CERT_DIR to 
This non-commercial license of GraphLab Create is assigned to 527536646@qq.com and will expire on October 23, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-1235 - Server binary: /usr/local/lib/python2.7/dist-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1452910312.log
[INFO] GraphLab Server Version: 1.7.1


In [4]:
from math import log,sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float)
sales['floors_square'] = sales['floors']*sales['floors']


# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [5]:
all_features = ['bedrooms','bedrooms_square','bathrooms',
                'sqft_living','sqft_living_sqrt',
                'sqft_lot','sqft_lot_sqrt',
                'floors','floors_square','waterfront','view','condition','grade',
               'sqft_above','sqft_basement',
               'yr_built','yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [6]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 21613
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000002  | 1.306641     | 6962915.603493     | 426631.749026 |
PROGRESS: | 2         | 3        | 0.000002  | 1.341030     | 6843144.200219     | 392488.929838 |
PROGRESS: | 3         | 4      

In [7]:
model_all.get("coefficients")

name,index,value
(intercept),None,274873.05595
bedrooms,None,0.0
bedrooms_square,None,0.0
bathrooms,None,8468.53108691
sqft_living,None,24.4207209824
sqft_living_sqrt,None,350.060553386
sqft_lot,None,0.0
sqft_lot_sqrt,None,0.0
floors,None,0.0
floors_square,None,0.0


In [8]:
for i in range(18):
    if model_all.get("coefficients")[i]['value']!=0:
        print model_all.get("coefficients")[i]

{'index': None, 'name': '(intercept)', 'value': 274873.0559504954}
{'index': None, 'name': 'bathrooms', 'value': 8468.53108691012}
{'index': None, 'name': 'sqft_living', 'value': 24.42072098244649}
{'index': None, 'name': 'sqft_living_sqrt', 'value': 350.0605533860531}
{'index': None, 'name': 'grade', 'value': 842.0680348975135}
{'index': None, 'name': 'sqft_above', 'value': 20.02472241709075}


Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [9]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [10]:
import numpy as np

l1_penalty =  np.logspace(1, 7, num=13)

In [11]:
print l1_penalty

[  1.00000000e+01   3.16227766e+01   1.00000000e+02   3.16227766e+02
   1.00000000e+03   3.16227766e+03   1.00000000e+04   3.16227766e+04
   1.00000000e+05   3.16227766e+05   1.00000000e+06   3.16227766e+06
   1.00000000e+07]


In [12]:
model = [0]*len(l1_penalty)
for i in range(len(l1_penalty)):
    model[i] = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty[i])

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 9761
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000003  | 0.114861     | 6287414.310279     | 322429.554621 |
PROGRESS: | 2         | 3        | 0.000003  | 0.129054     | 6004129.870008     | 303352.364502 |
PROGRESS: | 3         | 4       

In [13]:
def get_residual_sum_of_square(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)
    # Then compute the residuals/errors
    residuals = predictions - outcome
    # Then square and add them up
    RSS = (residuals*residuals).sum()
    return(RSS) 

In [14]:
for i in range(len(l1_penalty)):
    RSS = get_residual_sum_of_square(model[i], validation, validation['price'])
    print RSS
    if i == 0:
        least_rss = RSS
        NUM = 0
    elif least_rss > RSS:
        least_rss = RSS
        NUM = i
        
print least_rss,NUM
    

6.25766285142e+14
6.25766285362e+14
6.25766286058e+14
6.25766288257e+14
6.25766295212e+14
6.25766317206e+14
6.25766386761e+14
6.25766606749e+14
6.25767302792e+14
6.25769507644e+14
6.25776517727e+14
6.25799062845e+14
6.25883719085e+14
6.25766285142e+14 0


*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [15]:
print "1. 6.25766285142e+14"
RSS = get_residual_sum_of_square(model[0], testing, testing['price'])
print "2." 
print RSS

1. 6.25766285142e+14
2.
1.56983602382e+14


***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [16]:
NUM = 0
for i in range(18):
    if model[0].get("coefficients")[i]['value']!=0:
        print model[0].get("coefficients")[i]
        NUM += 1
print NUM

{'index': None, 'name': '(intercept)', 'value': 18993.427212770603}
{'index': None, 'name': 'bedrooms', 'value': 7936.967679031267}
{'index': None, 'name': 'bedrooms_square', 'value': 936.9933681933006}
{'index': None, 'name': 'bathrooms', 'value': 25409.58893412067}
{'index': None, 'name': 'sqft_living', 'value': 39.11513637970744}
{'index': None, 'name': 'sqft_living_sqrt', 'value': 1124.6502128077152}
{'index': None, 'name': 'sqft_lot', 'value': 0.003483618222990274}
{'index': None, 'name': 'sqft_lot_sqrt', 'value': 148.25839101140795}
{'index': None, 'name': 'floors', 'value': 21204.3354669502}
{'index': None, 'name': 'floors_square', 'value': 12915.524336072429}
{'index': None, 'name': 'waterfront', 'value': 601905.5945452707}
{'index': None, 'name': 'view', 'value': 93312.8573118719}
{'index': None, 'name': 'condition', 'value': 6609.035712447192}
{'index': None, 'name': 'grade', 'value': 6206.9399918805775}
{'index': None, 'name': 'sqft_above', 'value': 43.28705341933581}
{'inde

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [17]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [18]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [19]:
model2 = [0]*len(l1_penalty_values)
for i in range(len(l1_penalty_values)):
    model2[i] = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=l1_penalty_values[i])

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 9761
PROGRESS: Number of features          : 17
PROGRESS: Number of unpacked features : 17
PROGRESS: Number of coefficients    : 18
PROGRESS: Starting Accelerated Gradient (FISTA)
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Step size | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+-----------+--------------+--------------------+---------------+
PROGRESS: Tuning step size. First iteration could take longer than subsequent iterations.
PROGRESS: | 1         | 2        | 0.000003  | 0.114515     | 6302596.887972     | 323993.114447 |
PROGRESS: | 2         | 3        | 0.000003  | 0.129410     | 6023837.209073     | 303975.326597 |
PROGRESS: | 3         | 4       

In [20]:
for i in range(len(l1_penalty_values)):
    print model2[i]['coefficients']['value'].nnz()
    if model2[i]['coefficients']['value'].nnz() > max_nonzeros:
        if model2[i+1]['coefficients']['value'].nnz() < max_nonzeros:
            l1_penalty_min = l1_penalty_values[i]
            l1_penalty_max = l1_penalty_values[i+1]
            print l1_penalty_min, l1_penalty_max 

18
18
18
18
17
17
17
17
17
16
15
15
13
12
10
2976351441.63 3792690190.73
6
5
3
1
1


***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [21]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [23]:
print l1_penalty_values[0]

2976351441.63


In [37]:
ii = 0
for i in l1_penalty_values:
    model3 = graphlab.linear_regression.create(training, target='price', features=all_features, validation_set=None, l2_penalty=0., l1_penalty=i,verbose=0)
    if model3['coefficients']['value'].nnz() == max_nonzeros:
        RSS = get_residual_sum_of_square(model3, validation, validation['price'])
        print RSS
        if ii == 0:
            least_rss = RSS
            coefficient = model3['coefficients']
            l1_penal = i 
            num = 0
        elif least_rss > RSS:
            least_rss = RSS
            coefficient = model3['coefficients']
            l1_penal = i
            num = ii
        ii = ii + 1        
print least_rss, num, l1_penal
NUM = 0
for i in range(18):
    if coefficient[i]['value']!=0:
        print coefficient[i]
        NUM += 1
print NUM

1.04693748875e+15
1.05114762561e+15
1.05599273534e+15
1.06079953176e+15
1.04693748875e+15 0 3448968612.16
{'index': None, 'name': '(intercept)', 'value': 222253.1925443279}
{'index': None, 'name': 'bedrooms', 'value': 661.7227177821262}
{'index': None, 'name': 'bathrooms', 'value': 15873.957259267974}
{'index': None, 'name': 'sqft_living', 'value': 32.410221451259254}
{'index': None, 'name': 'sqft_living_sqrt', 'value': 690.1147733133093}
{'index': None, 'name': 'grade', 'value': 2899.4202697499395}
{'index': None, 'name': 'sqft_above', 'value': 30.01157530220153}
7


***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?